In [14]:
import pandas as pd
import os

DATA_DIR = "../data/raw"
CSV_FILE = os.path.join(DATA_DIR, "Agregados_por_setores_basico_BR_20250417.csv")
DICT_FILE = os.path.join(DATA_DIR, "dicionario_de_dados_agregados_por_setores_censitarios_20250417.xlsx")

# Verificar se arquivos existem
print(f"CSV existe: {os.path.exists(CSV_FILE)}")
print(f"Dicionário existe: {os.path.exists(DICT_FILE)}")

CSV existe: True
Dicionário existe: True


In [15]:
# Carregar Dicionário
# O dicionário costuma ter abas. Vamos listar as abas.
xls = pd.ExcelFile(DICT_FILE)
print("Abas do Excel:", xls.sheet_names)

# Geralmente a aba com as variáveis é algo como 'Basico' ou 'Descrição'
# Vamos carregar a primeira aba para ver
df_dict = pd.read_excel(DICT_FILE, sheet_name=0)
df_dict.head()

Abas do Excel: ['Dicionário Básico', 'Siglas Básico', 'Dicionário não PCT', 'Dicionário PCT - Indígenas', 'Dicionário PCT - Quilombolas']


,Tema,Variável,Descrição
0,Básico,V0001,Total de pessoas
1,Básico,V0002,Total de Domicílios (DPPO + DPPV + DPPUO + DPI...
2,Básico,V0003,Total de Domicílios Particulares (DPPO + DPPV ...
3,Básico,V0004,Total de Domicílios Coletivos (DCCM + DCSM)
4,Básico,V0005,Média de moradores em Domicílios Particulares ...


In [3]:
# Carregar uma amostra do CSV para identificar separadores e encoding
# IBGE costuma usar ';' e encoding 'latin1' ou 'utf-8'
try:
    df_sample = pd.read_csv(CSV_FILE, sep=';', nrows=100, encoding='utf-8')
except UnicodeDecodeError:
    print("UTF-8 falhou, tentando Latin-1")
    df_sample = pd.read_csv(CSV_FILE, sep=';', nrows=100, encoding='latin1')

df_sample.head()

UTF-8 falhou, tentando Latin-1


,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,...,NM_RGI,CD_CONCURB,NM_CONCURB,v0001,v0002,v0003,v0004,v0005,v0006,v0007
0,110001505000002,Urbana,1,0,"0,5393102",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,928,376,376,0,"2,8","0,0923",336
1,110001505000003,Urbana,1,0,"0,2362175",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,556,243,243,0,"2,7","0,0048",208
2,110001505000004,Urbana,1,0,"0,2118666",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,222,102,102,0,"2,6","0,0000",85
3,110001505000006,Urbana,1,0,"0,5054477",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,785,318,318,0,"2,8","0,0000",282
4,110001505000007,Urbana,1,0,"0,2990424",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,748,334,334,0,"2,6","0,0447",291


In [4]:
# Ver colunas
print(f"Total de colunas: {len(df_sample.columns)}")
print(df_sample.columns.tolist()[:20]) # Primeiras 20 colunas

Total de colunas: 36
['CD_SETOR', 'SITUACAO', 'CD_SIT', 'CD_TIPO', 'AREA_KM2', 'CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF', 'CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST', 'CD_BAIRRO', 'NM_BAIRRO', 'CD_NU', 'NM_NU', 'CD_FCU']


In [16]:
# Vamos tentar carregar o dataset completo (130MB é tranquilo)
# Definindo tipos para economizar memória se necessário, mas para 130MB o pandas aguenta default.
# Importante: CD_SETOR deve ser string para não perder zeros à esquerda, se houver.

df_full = pd.read_csv(CSV_FILE, sep=';', encoding='latin1', dtype={'CD_SETOR': str, 'CD_MUN': str, 'CD_UF': str})
df_full.info()

C:\Users\lfrpo\AppData\Local\Temp\ipykernel_16492\1686099976.py:5: DtypeWarning: Columns (2,3,11,13,14,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(CSV_FILE, sep=';', encoding='latin1', dtype={'CD_SETOR': str, 'CD_MUN': str, 'CD_UF': str})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468099 entries, 0 to 468098
Data columns (total 36 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   CD_SETOR    468099 non-null  object
 1   SITUACAO    466996 non-null  object
 2   CD_SIT      468099 non-null  object
 3   CD_TIPO     468099 non-null  object
 4   AREA_KM2    468099 non-null  object
 5   CD_REGIAO   468099 non-null  int64 
 6   NM_REGIAO   468099 non-null  object
 7   CD_UF       468099 non-null  object
 8   NM_UF       468099 non-null  object
 9   CD_MUN      468099 non-null  object
 10  NM_MUN      468097 non-null  object
 11  CD_DIST     468099 non-null  object
 12  NM_DIST     468097 non-null  object
 13  CD_SUBDIST  468099 non-null  object
 14  NM_SUBDIST  70874 non-null   object
 15  CD_BAIRRO   468099 non-null  object
 16  NM_BAIRRO   168050 non-null  object
 17  CD_NU       468099 non-null  object
 18  NM_NU       12338 non-null   object
 19  CD_FCU      468099 non-

In [6]:
# Cruzando com o dicionário para entender o que é V0001, V0002...
# Supondo que o dicionário tenha colunas 'Nome da Variável' e 'Descrição'

# Vamos ver a estrutura do dicionário novamente com mais detalhes
df_dict.head(10)

,Tema,Variável,Descrição
0,Básico,V0001,Total de pessoas
1,Básico,V0002,Total de Domicílios (DPPO + DPPV + DPPUO + DPI...
2,Básico,V0003,Total de Domicílios Particulares (DPPO + DPPV ...
3,Básico,V0004,Total de Domicílios Coletivos (DCCM + DCSM)
4,Básico,V0005,Média de moradores em Domicílios Particulares ...
5,Básico,V0006,Percentual de Domicílios Particulares Ocupados...
6,Básico,V0007,Total de Domicílios Particulares Ocupados (DPP...


In [17]:
# Tratamento: Renomear colunas para nomes legíveis
# Vamos criar um dicionário de-para: { 'V0001': 'Total de pessoas', ... }

# Limpar a descrição para usar como nome de coluna (remover caracteres especiais, encurtar)
import re
from unicodedata import normalize

def slugify(text):
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = text.lower()
    text = re.sub(r'[^a-z0-9]+', '_', text)
    text = text.strip('_')
    return text

# Criar mapa
# Importante: O CSV tem colunas em minúsculo (v0001), mas o dicionário está em maiúsculo (V0001)
# Vamos normalizar as chaves para minúsculo
var_map = dict(zip(df_dict['Variável'].str.lower(), df_dict['Descrição']))

# Aplicar slugify
var_map_slug = {k: slugify(v) for k, v in var_map.items()}

# Mostrar alguns exemplos
list(var_map_slug.items())[:5]

[('v0001', 'total_de_pessoas'),
 ('v0002', 'total_de_domicilios_dppo_dppv_dppuo_dpio_dccm_dcsm'),
 ('v0003', 'total_de_domicilios_particulares_dppo_dppv_dppuo_dpio'),
 ('v0004', 'total_de_domicilios_coletivos_dccm_dcsm'),
 ('v0005',
  'media_de_moradores_em_domicilios_particulares_ocupados_total_pessoas_em_domicilios_particulares_ocupados_dppo_dpio')]

In [18]:
# Renomear no dataframe
df_treated = df_full.rename(columns=var_map_slug)
df_treated.head()

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_de_pessoas,total_de_domicilios_dppo_dppv_dppuo_dpio_dccm_dcsm,total_de_domicilios_particulares_dppo_dppv_dppuo_dpio,total_de_domicilios_coletivos_dccm_dcsm,media_de_moradores_em_domicilios_particulares_ocupados_total_pessoas_em_domicilios_particulares_ocupados_dppo_dpio,percentual_de_domicilios_particulares_ocupados_imputados_total_dpo_imputados_total_dpo,total_de_domicilios_particulares_ocupados_dppo_dpio
0,110001505000002,Urbana,1,0,"0,5393102",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,928,376,376,0,"2,8","0,0923",336
1,110001505000003,Urbana,1,0,"0,2362175",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,556,243,243,0,"2,7","0,0048",208
2,110001505000004,Urbana,1,0,"0,2118666",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,222,102,102,0,"2,6","0,0000",85
3,110001505000006,Urbana,1,0,"0,5054477",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,785,318,318,0,"2,8","0,0000",282
4,110001505000007,Urbana,1,0,"0,2990424",1,Norte,11,Rondônia,1100015,...,Cacoal,.,NaN,748,334,334,0,"2,6","0,0447",291


In [11]:
# Verificando valores nulos (Quebras)
missing = df_treated.isnull().sum()
missing = missing[missing > 0]
print("Colunas com valores nulos:")
print(missing)

# Porcentagem de nulos
print("\nPorcentagem de nulos:")
print((missing / len(df_treated)) * 100)

Colunas com valores nulos:
SITUACAO        1103
NM_MUN             2
NM_DIST            2
NM_SUBDIST    397225
NM_BAIRRO     300049
NM_NU         455761
NM_FCU        434778
NM_AGLOM      436893
NM_RGINT           2
NM_RGI             2
NM_CONCURB    207749
dtype: int64

Porcentagem de nulos:
SITUACAO       0.235634
NM_MUN         0.000427
NM_DIST        0.000427
NM_SUBDIST    84.859186
NM_BAIRRO     64.099475
NM_NU         97.364233
NM_FCU        92.881634
NM_AGLOM      93.333462
NM_RGINT       0.000427
NM_RGI         0.000427
NM_CONCURB    44.381424
dtype: float64


In [23]:
# Tratamento de valores especiais
# IBGE usa '.' para valores nulos ou não aplicáveis em algumas colunas numéricas
import numpy as np

# Substituir '.' por NaN
df_treated = df_treated.replace('.', np.nan)

# Normalizar CD_SIT e outras colunas de código que podem estar mistas
# CD_SIT parece ter int e str misturados. Vamos converter para numérico onde possível.
cols_to_numeric = ['CD_SIT', 'CD_TIPO'] # Adicionar outros se necessário
for col in cols_to_numeric:
    if col in df_treated.columns:
        df_treated[col] = pd.to_numeric(df_treated[col], errors='coerce')

# Tentar converter colunas numéricas que estavam como object
for col in df_treated.columns:
    if df_treated[col].dtype == 'object':
        try:
            if col.startswith('CD_'):
                continue # Manter códigos como string ou int, mas cuidado
                
            # Se for variável de valor (v0001...), deve ser numérico
            if df_treated[col].str.contains(',', na=False).any():
                 df_treated[col] = df_treated[col].str.replace(',', '.').astype(float)
            else:
                 df_treated[col] = pd.to_numeric(df_treated[col], errors='ignore')
                 
        except Exception as e:
            pass

# Verificar tipos novamente
df_treated.info()

C:\Users\lfrpo\AppData\Local\Temp\ipykernel_16492\3554249192.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_treated[col] = pd.to_numeric(df_treated[col], errors='ignore')
C:\Users\lfrpo\AppData\Local\Temp\ipykernel_16492\3554249192.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_treated[col] = pd.to_numeric(df_treated[col], errors='ignore')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468099 entries, 0 to 468098
Data columns (total 36 columns):
 #   Column                                                                                                              Non-Null Count   Dtype  
---  ------                                                                                                              --------------   -----  
 0   CD_SETOR                                                                                                            468099 non-null  object 
 1   SITUACAO                                                                                                            466996 non-null  object 
 2   CD_SIT                                                                                                              468097 non-null  float64
 3   CD_TIPO                                                                                                             468097 non-null  float64
 4   AREA_KM2

In [25]:
# Persistir em Parquet (Camada Silver)
OUTPUT_DIR = "../data/silver"
os.makedirs(OUTPUT_DIR, exist_ok=True)
PARQUET_FILE = os.path.join(OUTPUT_DIR, "censo_2022_basico_tratado.parquet")

# Usando fastparquet para evitar conflito de versões do pyarrow
df_treated.to_parquet(PARQUET_FILE, index=False, engine='fastparquet')
print(f"Salvo em {PARQUET_FILE}")

Salvo em ../data/silver\censo_2022_basico_tratado.parquet


## Conclusão
O dataset "Básico" foi carregado, tratado e salvo em formato Parquet na pasta `data/silver`.
As colunas foram renomeadas para facilitar o uso e os tipos de dados foram corrigidos.
Próximos passos:
1. Carregar outros datasets (Demografia, etc.) e fazer o merge pelo `CD_SETOR`.
2. Baixar a malha de setores (geobr) e fazer o join espacial.